In [30]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import scipy.io
import glob
import os
from math import sqrt
import json
import torch
import torch.nn as nn
import torchvision
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import pathlib

In [31]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [44]:
batch_size = 16
lr = 0.001
epochs = 10

In [57]:
training_dir = 'training_vacancy/'
train_count=len(glob.glob(training_dir+'/**/*.jpg'))

In [58]:
transformer = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [59]:
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(training_dir,transform = transformer),
    batch_size = 128,
    shuffle = True
)

In [60]:
classes = ['vacant','non-vacant','parking']

In [61]:
class VPSNet(nn.Module):
    def __init__(self,num_classes = 3):
        super(VPSNet,self).__init__()
        
        self.model = nn.Sequential(
            nn.Conv2d(3, 40, kernel_size=(3, 9), stride=(1, 2)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=2),
            nn.Conv2d(40, 80, kernel_size=(3, 5), padding=(1, 0)),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=(1, 0)),
            nn.Conv2d(80, 120, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(120, 160, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Flatten(),
            nn.Dropout(),
            nn.Linear(160 * 5 * 5, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, num_classes)
        )
    def forward(self,x):
        x = self.model(x)
        return x

In [62]:
model = VPSNet(num_classes=3).to(device)

In [63]:
optimizer = Adam(model.parameters(),lr = lr,betas=(0.9,0.999),eps = 1e-08,weight_decay=0.0001)
loss_fn = nn.CrossEntropyLoss()

In [64]:
for epoch in range(epochs):
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    
    for i,(images,labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        optimizer.zero_grad()
        loss = loss_fn
        optimizer.zero_grad()
        outputs = model(images)
        loss = loss_fn(outputs,labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.cpu().data*images.size(0)
        _,prediction = torch.max(outputs.data,1)
        train_accuracy += int(torch.sum(prediction == labels.data))
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
    
    print('epoch : {} , train loss: {} , train accuracy : {}'.format(epoch,train_loss,train_accuracy))

epoch : 0 , train loss: 0.5035967230796814 , train accuracy : 0.807753164556962
epoch : 1 , train loss: 0.2619050443172455 , train accuracy : 0.9063818565400844
epoch : 2 , train loss: 0.18580277264118195 , train accuracy : 0.9371044303797469
epoch : 3 , train loss: 0.1338103711605072 , train accuracy : 0.9583333333333334
epoch : 4 , train loss: 0.12017528712749481 , train accuracy : 0.9621571729957806
epoch : 5 , train loss: 0.08319012075662613 , train accuracy : 0.9757383966244726
epoch : 6 , train loss: 0.10105359554290771 , train accuracy : 0.9713871308016878
epoch : 7 , train loss: 0.08055329322814941 , train accuracy : 0.9753428270042194
epoch : 8 , train loss: 0.06636925786733627 , train accuracy : 0.9814082278481012
epoch : 9 , train loss: 0.07214481383562088 , train accuracy : 0.9777162447257384
